In [8]:
import pandas as pd

In [9]:
accounts = pd.read_csv("data/accounts.csv")
alliance_ff = pd.read_csv("data/alliance_ff.csv")
alliance_membership = pd.read_csv("data/alliance_membership.csv")
messages_1 = pd.read_csv("data/chat_messages_1.csv")
messages_2 = pd.read_csv("data/chat_messages_2.csv")

/Users/giomonopoli/miniconda3/envs/adaexam/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (6,8,11,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/giomonopoli/miniconda3/envs/adaexam/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,10,12,13,15,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
messages_2 = pd.read_csv("data/cleaned_messages_2.csv")

In [16]:
#divide messages_2.groupby('account_id')['timestamp'] \
                             #.count() by the number of entries qwithout ending up with AttributeError: 'int' object has no attribute 'reset_index'
messages_2.groupby('account_id')['timestamp'] \
                                .count() / messages_2.groupby('account_id')['timestamp'] \
                                .count().count()

account_id
00009fe7185adef70c1a19c70cba655dc0cf0e5304d50f3af14f941086015440    0.000005
00025a45a6ff5c2035ac526bd5d9395938beaac708778f9c963eada62e668ee4    0.000005
0002d70ca9f02320c4b873884a8cf78f164ea04741c7556687044277509a12a1    0.000005
0003fc20d36037e6033c16116e49e8b5ce22b86e4d3f33129e6241987d0de354    0.000005
00046197b9df4fbee080a31b0a8b4945f679c418220f89ef64463a49b7b1bcf2    0.000011
                                                                      ...   
fffb33636b5f5fa363c7d333e63bcd6f1c9c54b97105feda82ddcfe9fa3d6171    0.000005
fffb4d64336538c43e04fdc1ad7b3a66efa42654947e13d472d83131659ba88e    0.000005
fffd7747f61326301a76c64374ad3b34fd5957e5ef7c7b5c37800c46fc5ac6e2    0.000005
fffe6519257b695dfa86b12e2bf5deb227db39e5eddafdf80bd52f3cea850273    0.000011
ffffd47c89d917033c8e5150545a9e1caca84f64a1ca309845a3bee52f2930b5    0.000005
Name: timestamp, Length: 83805, dtype: float64

In [55]:
mean_messages = messages_2.groupby('account_id')['timestamp'].count().mean()

In [56]:
percentages["count"]

0        1
50493    1
50494    1
50495    1
50497    1
Name: count, dtype: int64

In [57]:
#find the users with the lowest interaction
percentages = messages_2.groupby('account_id')['timestamp'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=True) \
                             .head(5)
low_interaction_users = percentages[percentages["count"] < mean_messages].account_id

In [58]:
#find the alliances of the users with the lowest interaction
allies_low_interaction_users = messages_2[messages_2["account_id"].isin(low_interaction_users.unique())]["alliance_id"]

In [60]:
messages_2["GENERAL_RISK"]

0         1.0
1         2.0
2         2.0
3         3.0
4         2.0
         ... 
182782    1.0
182783    1.0
182784    2.0
182785    1.0
182786    2.0
Name: GENERAL_RISK, Length: 182787, dtype: float64

In [63]:
#get the risky allies
risky_allies = messages_2[messages_2["GENERAL_RISK"] > 1.0].alliance_id.unique()

In [68]:
low_interaction_users

0        00009fe7185adef70c1a19c70cba655dc0cf0e5304d50f...
50484    9a299069152b6bb13a1712b16ae8c25af70f2f6a5b28a4...
50486    9a2b8169998a2ed97a2147c2c586ad734805c44b554dde...
50487    9a2e1d60da91bbee0a2faf4cd3f58ad9059b66ceb163ec...
50488    9a304ad2233dcccd06c4cfee5d615c686ae38b00636ba1...
Name: account_id, dtype: object

In [70]:
#create a function that checks if the value is both on the lists and if so return the value
matches_found = [x for x in allies_low_interaction_users if x in risky_allies]
#get the user from messages_2 that are in low_interaction_users that have the alliance id in matches_found
affected_users = messages_2[messages_2["account_id"].isin(low_interaction_users.unique()) & messages_2["alliance_id"].isin(matches_found)]["account_id"]

In [71]:
affected_users

95759     9a304ad2233dcccd06c4cfee5d615c686ae38b00636ba1...
116241    00009fe7185adef70c1a19c70cba655dc0cf0e5304d50f...
164979    9a2e1d60da91bbee0a2faf4cd3f58ad9059b66ceb163ec...
Name: account_id, dtype: object

In [43]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-offensive")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-offensive")

# define the input texts
input_texts = messages_2[""]

# tokenize the input texts
tokens = tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt")

# perform inference
outputs = model(**tokens)

# get the predicted labels
predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()

# print the predicted labels
for i, text in enumerate(input_texts):
    print("Input text:", text)
    print("Predicted label:", predicted_labels[i])

Downloading: 100%|██████████| 725/725 [00:00<00:00, 134kB/s]
Downloading:  95%|█████████▌| 836k/878k [00:49<00:01, 22.5kB/s] 

KeyboardInterrupt: 

In [65]:
accounts

,account_id,alliance_id,session_count,session_duration,transaction_count,revenue,date,active,account_state,last_active_date,level,created_language,created_country_code,created_time,session_count_today,session_duration_today,transaction_count_today,revenue_today,last_login_game_client_language,last_login_country_code
0,e10cd6ddf379b98c52e59529026359ad80e53ebecd38cd...,7f87a7d72c4038b8845735c772a0407cb58d4a443508d1...,492,1.197445e+06,0,0.00,2023-03-01,True,3,2023-03-01,33.0,en-GB,IN,2022-06-30 09:57:13.976,2,5275.434000,0,0.0,en-IN,IN
1,592103594bc6e660e1d0caa9e046d7b7f6da3eede4e893...,beb86b8e68696fb10c3b3c55898a495568579cc17e2eb4...,536,4.138838e+05,0,0.00,2023-03-01,True,3,2023-03-01,31.0,en-GB,IN,2021-11-26 14:20:32.716,3,1691.798000,0,0.0,en-IN,IN
2,7be671feb8066897a6292e03e520aa4ffbc9cae00ffe99...,61b3a23ded0bdd6d3df3d0d7f741af2eb48e6f9bc662dc...,1115,5.606682e+05,0,0.00,2023-03-01,True,3,2023-03-01,32.0,en-GB,IN,2021-06-30 09:22:12.642,14,4192.960001,0,0.0,en-IN,IN
3,0cea88f4366301f6c58f45d131180aeb6108df7820f3cc...,a4979799f92bcec14255788a66d24be68de82c8afa1252...,2076,1.303385e+06,0,0.00,2023-03-01,True,3,2023-03-01,38.0,en-GB,IN,2021-08-20 06:40:01.696,3,5331.243000,0,0.0,en-GB,IN
4,57eb9194ccc208704bf991dd56a5cb3fff63c9b0ce61f6...,546c74e081f1d793ad223ac6ac4cedf226b0a4d285fae2...,1908,1.461803e+06,0,0.00,2023-03-01,True,3,2023-03-01,35.0,en-GB,IN,2022-05-26 13:10:14.224,10,3949.426000,0,0.0,en-GB,IN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130132,0b3dc5e1e27efe270d4efc35d0c45431ea776c88dc0765...,b981c6e1505f98aece038f634159bcb88fe4165a6f8436...,1455,9.503963e+05,2,9.98,2023-03-01,True,3,2023-03-01,29.0,en-CA,CA,2018-03-04 03:20:04.025,1,795.201000,0,0.0,en-CA,CA
130133,514ec922ee48e22eb15084fab3cbf88f817e4030edec05...,569c41f38d9d1fd8a421615b54edc58ad09d4ecc554da9...,15029,5.437979e+06,1,0.99,2023-03-01,True,3,2023-03-01,50.0,en-CA,CA,2018-12-16 19:29:01.851,9,3150.533000,0,0.0,en-CA,CA
130134,52837ef4e11812234cf93ead2d84eef2ce7cd881e915a4...,6f09682eb5d3c6bbe3a7bee1f35e594da10f0565260362...,2128,9.727380e+05,6,21.94,2023-03-01,True,3,2023-03-01,35.0,es-ES,ES,2019-07-02 13:53:03.668,23,16816.713000,0,0.0,es-ES,ES
130135,da1afdf5015045c8875c33d7cef8c8be8603c0ea024af7...,ebcf7976b86d091e46a503ad0767e1dcd2490eb4f81bf2...,2212,1.247516e+06,0,0.00,2023-03-01,True,3,2023-03-01,34.0,es-US,MX,2022-04-28 01:27:44.066,5,1826.763000,0,0.0,es-US,MX


In [73]:
family_friendly_alliances = alliance_ff[alliance_ff["family_friendly"]==1].alliance_id.unique()
non_family_friendly_alliances = alliance_ff[alliance_ff["family_friendly"]==-1].alliance_id.unique()

In [96]:
family_friendly_risk

9         1.0
11        2.0
27        1.0
30        0.0
31        1.0
         ... 
182738    1.0
182739    2.0
182762    1.0
182773    1.0
182783    1.0
Name: GENERAL_RISK, Length: 17128, dtype: float64

In [21]:
import matplotlib.pyplot as plt

# Get risk for family_friendly_alliances and non_family_friendly_alliances
family_friendly_risk = messages_2[messages_2["alliance_id"].isin(family_friendly_alliances)].GENERAL_RISK
non_family_friendly_risk = messages_2[messages_2["alliance_id"].isin(non_family_friendly_alliances)].GENERAL_RISK

plt.hist(family_friendly_risk, bins=100, density=True, alpha=0.5, label="Family Friendly", linewidth=2)
plt.hist(non_family_friendly_risk, bins=100, density=True, alpha=0.5, label="Non Family Friendly", linewidth=2)
plt.legend(loc="upper right")
plt.show()



NameError: name 'family_friendly_alliances' is not defined

In [11]:
import numpy as np
bad = messages_2[messages_2['risk'] >= 5.0]['raw_message'].values
good = messages_2[messages_2['risk'] <= 1.0]['raw_message'].iloc[0:10967].values
data = np.concatenate((bad,good),axis=0)
labels = np.concatenate((np.full(len(bad),1), np.full(len(bad),0)), axis=0)

In [105]:
data.dtype

dtype('O')

In [12]:
from sklearn.metrics import classification_report
from transformers import pipeline

# Load the offensive language detection pipeline
nlp = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-offensive")

# Use the pipeline to generate predictions for the inputs
predictions = nlp(data.tolist())

# Extract the predicted labels from the output dictionary
predicted_labels = [p['label'] for p in predictions]

# Print a classification report to evaluate model performance
print(classification_report(labels, predicted_labels))

KeyboardInterrupt: 

In [7]:
predicted_labels

[0, 1, 1, 1, 0, 0, 1, 0, 0, 1]

In [17]:
data[:10]

array(['half ur deck is level 10 gtfo🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣',
       'As a hog player he was shit', 'Good shit slayer',
       'Miner poison is so fucking broken', 'Penis envy?',
       'this appears to be my willy!', 'I’m ass now',
       'Aryan toh topper bomcha hain', 'Peeee',
       "now I bet it's half that shit"], dtype=object)

In [44]:
len(labels)

21934

In [34]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import classification_report

# Load the model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-offensive"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the inputs
tokenized_inputs = tokenizer(['gg'], padding=True, truncation=True, return_tensors="pt")

# Generate predictions for the inputs
with torch.no_grad():
    logits = model(tokenized_inputs['input_ids'], tokenized_inputs['attention_mask'])[0]
    print(logits)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


tensor([[ 0.4413, -0.3595]])


In [41]:
#shuffle labels and data the same way
from sklearn.utils import shuffle
data, labels = shuffle(data, labels)

In [42]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import classification_report

# Load the model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-offensive"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the inputs
tokenized_inputs = tokenizer(data[:1000].tolist(), padding=True, truncation=True, return_tensors="pt")

# Generate predictions for the inputs
with torch.no_grad():
    logits = model(tokenized_inputs['input_ids'], tokenized_inputs['attention_mask'])[0]
    predicted_labels = torch.argmax(logits, axis=1).tolist()

# Print a classification report to evaluate model performance
print(classification_report(labels[:1000], predicted_labels))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyboardInterrupt: 

**Data visualisation**
- How is the risk distributed across ages (family-friendly vs non family friendly)?
- 